<a href="https://colab.research.google.com/github/Ryuga-17/Sentiment_analysis/blob/main/new_pip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sbhatti_financial_sentiment_analysis_path = kagglehub.dataset_download('sbhatti/financial-sentiment-analysis')

print('Data source import complete.')


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df=pd.read_csv('/content/data (1).csv')

In [4]:
df.isnull().sum()

,0
Sentence,0
Sentiment,0


In [5]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [6]:
hf=df.copy()

In [7]:
hf['Sentiment'].replace(['positive','neutral','negative'],[0,1,2],inplace=True)

<ipython-input-7-25283a87dac6>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hf['Sentiment'].replace(['positive','neutral','negative'],[0,1,2],inplace=True)
<ipython-input-7-25283a87dac6>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hf['Sentiment'].replace(['positive','neutral','negative'],[0

In [8]:
hf['Sentiment']

,Sentiment
0,0
1,2
2,0
3,1
4,1
...,...
5837,2
5838,1
5839,1
5840,1


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [11]:
rf=RandomForestClassifier()
xg=XGBClassifier()

In [12]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [13]:
stemmer=PorterStemmer()

In [14]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
def stemming(con):
    content=re.sub("[^a-zA-z]"," ",con)
    content=content.lower()
    content=content.split()
    content=[stemmer.stem(word) for word in content if not word in stopwords.words('english')]
    content=' '.join(content)
    return content


In [27]:
#converting the above function to a class that can then be applied through pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class StemmingTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
      self.stemmer=PorterStemmer()

    def fit(self,X,y=None):
      return self

    def transform(self,X,y=None):
      X = X.astype(str)
      return X.apply(self.stemming)

    def stemming(self,con):
      content=re.sub("[^a-zA-z]"," ",con)
      content=content.lower()
      content=content.split()
      content = [self.stemmer.stem(word) for word in content if not word in stopwords.words('english')]
      content = ' '.join(content)
      return content





In [16]:
hf['Sentence']=hf['Sentence'].apply(stemming)

In [17]:
hf['Sentence']

,Sentence
0,geosolut technolog leverag benefon gp solut pr...
1,esi low bk real possibl
2,last quarter componenta net sale doubl eur eur...
3,accord finnish russian chamber commerc major c...
4,swedish buyout firm sold remain percent stake ...
...,...
5837,rise cost forc packag produc huhtamaki axe job...
5838,nordic walk first use summer train method cros...
5839,accord ship compani vike line eu decis signifi...
5840,build home improv trade sale decreas eur mn


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cv=CountVectorizer(ngram_range=(1,1))
u=cv.fit_transform(hf['Sentence'])
v=hf['Sentiment']

In [20]:
u_train,u_test,v_train,v_test=train_test_split(u,v,test_size=0.05,random_state=82)

In [21]:

xg.fit(u_train,v_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
xg.score(u_test,v_test)

0.7064846416382252

In [24]:
from sklearn.pipeline import Pipeline

In [28]:
steps=([('stemming',StemmingTransformer()),('cv',CountVectorizer(ngram_range=(1,1))),('xg',XGBClassifier())])


In [30]:
pi=Pipeline(steps)

In [32]:
pi.fit(hf['Sentence'],hf['Sentiment'])

Pipeline(steps=[('stemming', StemmingTransformer()), ('cv', CountVectorizer()),
                ('xg',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constrain